<a href="https://colab.research.google.com/github/taehyeonk/pre-onboarding_project/blob/main/%EA%B8%B0%EC%97%85%EA%B3%BC%EC%A0%9C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import pandas as pd
import numpy as np 
import re
from typing import List, Dict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math, copy, time
import random

In [2]:
!pip install transformers

In [3]:
!pip install datasets

In [4]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [5]:
# seed
seed = 7777
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [6]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/Colab Notebooks/klue-sts-v1.1

/content/drive/MyDrive/Colab Notebooks/klue-sts-v1.1


In [8]:
df=pd.read_json('klue-sts-v1.1_train.json', orient='records')
df.drop(columns='annotations',inplace=True)
df.head()

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['0..."
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...","{'agreement': '5:0:0:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1..."
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0....","{'agreement': '4:2:1:0:0:0', 'annotators': ['1..."
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1..."


In [7]:
from torch.utils.data import DataLoader
from datasets import load_dataset
datasets = load_dataset("klue", "sts")

Downloading:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("klue/roberta-large")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [11]:
from datetime import datetime
train_batch_size = 32
num_epochs = 4
model_name='klue/roberta-large'
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [15]:
from sentence_transformers import SentenceTransformer, losses, models, util
embedding_model = models.Transformer(model_name)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [16]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)
model = SentenceTransformer(modules=[embedding_model, pooler])

In [17]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 11668
    })
    validation: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 519
    })
})

In [18]:
train_samples = []
dev_samples = []
test_samples = []

# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], 
            label=score,
        )

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

In [19]:
train_samples[0].texts, train_samples[0].label

(['숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
  '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.'],
 0.74)

In [20]:
train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size,
)
train_loss = losses.CosineSimilarityLoss(model=model)

In [23]:
import logging
from sentence_transformers import  LoggingHandler
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
logging.info(f"Warmup-steps: {warmup_steps}")

2022-03-14 03:33:29 - Warmup-steps: 146


In [25]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev",
)

In [26]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2022-03-14 03:40:35 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2022-03-14 03:40:38 - Cosine-Similarity :	Pearson: 0.8765	Spearman: 0.8722
2022-03-14 03:40:38 - Manhattan-Distance:	Pearson: 0.8748	Spearman: 0.8711
2022-03-14 03:40:38 - Euclidean-Distance:	Pearson: 0.8750	Spearman: 0.8715
2022-03-14 03:40:38 - Dot-Product-Similarity:	Pearson: 0.8622	Spearman: 0.8560
2022-03-14 03:40:38 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-14_03-26-15


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2022-03-14 03:46:24 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 1:
2022-03-14 03:46:27 - Cosine-Similarity :	Pearson: 0.8817	Spearman: 0.8782
2022-03-14 03:46:27 - Manhattan-Distance:	Pearson: 0.8832	Spearman: 0.8793
2022-03-14 03:46:27 - Euclidean-Distance:	Pearson: 0.8830	Spearman: 0.8791
2022-03-14 03:46:27 - Dot-Product-Similarity:	Pearson: 0.8627	Spearman: 0.8556
2022-03-14 03:46:27 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-14_03-26-15


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2022-03-14 03:52:13 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 2:
2022-03-14 03:52:16 - Cosine-Similarity :	Pearson: 0.8901	Spearman: 0.8876
2022-03-14 03:52:16 - Manhattan-Distance:	Pearson: 0.8866	Spearman: 0.8841
2022-03-14 03:52:16 - Euclidean-Distance:	Pearson: 0.8864	Spearman: 0.8840
2022-03-14 03:52:16 - Dot-Product-Similarity:	Pearson: 0.8763	Spearman: 0.8715
2022-03-14 03:52:16 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-14_03-26-15


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2022-03-14 03:58:03 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:
2022-03-14 03:58:06 - Cosine-Similarity :	Pearson: 0.8902	Spearman: 0.8876
2022-03-14 03:58:06 - Manhattan-Distance:	Pearson: 0.8869	Spearman: 0.8846
2022-03-14 03:58:06 - Euclidean-Distance:	Pearson: 0.8870	Spearman: 0.8850
2022-03-14 03:58:06 - Dot-Product-Similarity:	Pearson: 0.8759	Spearman: 0.8712
2022-03-14 03:58:06 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-14_03-26-15


In [27]:
testsets = load_dataset("kor_nlu", "sts")
test_samples = []
# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')

Downloading:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/89.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/66.1k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset kor_nlu downloaded and prepared to /root/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

2022-03-14 03:58:48 - Load pretrained SentenceTransformer: output/training_klue_sts_klue-roberta-large-2022-03-14_03-26-15
2022-03-14 03:58:51 - Use pytorch device: cuda


In [28]:
test_evaluator(model, output_path=model_save_path)

2022-03-14 03:59:10 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2022-03-14 03:59:18 - Cosine-Similarity :	Pearson: 0.7847	Spearman: 0.7768
2022-03-14 03:59:18 - Manhattan-Distance:	Pearson: 0.7680	Spearman: 0.7731
2022-03-14 03:59:18 - Euclidean-Distance:	Pearson: 0.7677	Spearman: 0.7729
2022-03-14 03:59:18 - Dot-Product-Similarity:	Pearson: 0.7615	Spearman: 0.7540


0.7768287690436116

In [30]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0255,  0.0091, -0.0176,  ...,  0.0848, -0.0065, -0.0122],
         [-0.0220,  0.0202, -0.0613,  ...,  0.0057,  0.0158, -0.0293],
         [-0.0497, -0.0001,  0.0381,  ..., -0.0456,  0.0472, -0.0694],
         ...,
         [-0.0321,  0.0126,  0.0211,  ...,  0.1073,  0.0039, -0.0126],
         [-0.0529,  0.0171, -0.0152,  ...,  0.1045,  0.0158, -0.0414],
         [-0.0108,  0.0136,  0.0104,  ...,  0.1172, -0.0010, -0.0084]],
        device='cuda:0', requires_grad=True), Parameter containing:
 tensor([[ 0.0389, -0.0049, -0.0123,  ...,  0.0079,  0.0034, -0.0174],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0692,  0.0313, -0.0069,  ..., -0.0333, -0.0195,  0.0227],
         ...,
         [ 0.0013,  0.0204,  0.0062,  ...,  0.0740, -0.0112,  0.0134],
         [ 0.0080,  0.0531, -0.0291,  ...,  0.0516, -0.0156,  0.0072],
         [-0.0278,  0.0581,  0.0309,  ...,  0.0609, -0.0155,  0.0589]],
        device='cuda:0', re